In [2]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import sys
sys.path.append("..") 

In [10]:
train_data = pd.read_csv('Data/train.csv')
test_data = pd.read_csv('Data/test.csv')

In [13]:
train_data.shape,test_data.shape

((1460, 81), (1459, 80))

In [22]:
train_data.iloc[0:10, [0, 1, 2, 3, 4, 5, 6, -3, -2, -1]]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,NaN,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,NaN,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,NaN,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,NaN,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,NaN,WD,Normal,118000


In [16]:
# 将 训练集和测试集合合并在一起， 一起做数据预处理
all_features = pd.concat((train_data.iloc[:,1:-1], test_data.iloc[:,1:-1]))

# 数据预处理

In [24]:
# 对连续数值做 标准化处理
numerical_features = all_features.dtypes[all_features.dtypes != 'object'].index
all_features[numerical_features] = all_features[numerical_features].apply(lambda x : (x-x.mean()) / (x.std()))
# 标准化后 将NnN 用0填充
all_features[numerical_features] = all_features[numerical_features].fillna(0)

In [25]:
# 对离散数值 进行独热编码 可见 将特征扩展到了331
all_features = pd.get_dummies(all_features, dummy_na=True)
all_features.shape

(2919, 331)

In [26]:
# 将pandas 数据转为numpy再转换未tensor
n_train = train_data.shape[0] 
train_features = torch.tensor(all_features[0:n_train].values,dtype=torch.float)
test_features = torch.tensor(all_features[n_train:].values,dtype=torch.float)
train_labels = torch.tensor(train_data.SalePrice.values, dtype = torch.float).view(-1,1)

In [29]:
train_features.shape, train_labels.shape

(torch.Size([1460, 331]), torch.Size([1460, 1]))

# 训练模型

In [ ]:
loss = torch.nn.EMSLoss()

def get_net(featurn_num):
    net = torch.nn.Linear(featurn_num,1)
    for param in net.parameters():
        nn.init.normal_(param, mean=0, std=0.01)
    return net

In [32]:
# 定义比赛用的对数均方根误差
def log_rmse(net, features, labels):
    with torch.no_grad():
        # 将小于1的值设置为1，使得取对数是数值更稳定
        clipped_preds = torch.max(net(features),torch.tensor(1.0))
        rmse = torch.sqrt(loss(clipped_preds.log(), labels.log()))
    return rmse.item()